In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
#File 1 input
feedbackClean = pd.read_excel('feedback.xlsx')

#File 2 input (Separating sheets into individual dataframes)
xls1 = pd.ExcelFile('sales-and-feedback.xlsx')
customerFeedback = pd.read_excel(xls1, "Customer Feedback")
salesHistory = pd.read_excel(xls1, "Sales History")

#File 3 input (Separated by individual sheets)
xls2 = pd.ExcelFile('transaction-records.xlsx')
countryCodes = pd.read_excel(xls2, "Country_Codes")
geography = pd.read_excel(xls2, "Geography")
orders = pd.read_excel(xls2, "Orders")
orderItems = pd.read_excel(xls2, "Order_Items")
customer = pd.read_excel(xls2, "Customer")
product = pd.read_excel(xls2, "Product")

In [3]:
orders

,Seq,Order Number,Customer_ID,Order_Date,Ship_Date,Status,N_Items
0,1,1,310020,2019-01-01,2019-01-04,Complete,1
1,2,2,9400009,2019-01-01,2019-01-04,Complete,8
2,3,3,5300016,2019-01-01,2019-01-03,Complete,8
3,4,4,700002,2019-01-01,2019-01-04,Complete,2
4,5,5,5300016,2019-01-10,2019-01-12,Complete,6
...,...,...,...,...,...,...,...
95,96,96,3500018,2019-03-12,2019-03-15,Complete,3
96,97,97,7910010,2019-03-12,2019-03-15,Complete,3
97,98,98,9400009,2019-03-12,2019-03-14,Complete,4
98,99,99,7410014,2019-03-12,2019-03-14,Complete,5


In [4]:
orderItems

,Sequence,Order_Count,Order Number,N_Items,Order_Item,Status,Product_ID
0,1,1,1,1,1,Complete,103060209
1,2,2,2,8,1,Complete,201030402
2,3,2,2,8,2,Complete,103040203
3,4,2,2,8,3,Complete,101020503
4,5,2,2,8,4,Complete,101040401
...,...,...,...,...,...,...,...
495,496,113,113,8,5,Complete,103030904
496,497,113,113,8,6,Complete,103030904
497,498,113,113,8,7,Complete,103030804
498,499,113,113,8,8,Complete,301060704


In [5]:
mergeOrders = pd.merge(orders, orderItems, on="Order Number")

In [6]:
mergeOrders = mergeOrders.drop(["Sequence","Ship_Date","Status_x","Status_y","N_Items_y","Order_Count","Seq"], axis=1)

In [7]:
mergeOrders

,Order Number,Customer_ID,Order_Date,N_Items_x,Order_Item,Product_ID
0,1,310020,2019-01-01,1,1,103060209
1,2,9400009,2019-01-01,8,1,201030402
2,2,9400009,2019-01-01,8,2,103040203
3,2,9400009,2019-01-01,8,3,101020503
4,2,9400009,2019-01-01,8,4,101040401
...,...,...,...,...,...,...
439,99,7410014,2019-03-12,5,4,110010307
440,99,7410014,2019-03-12,5,5,110060408
441,100,7410014,2019-03-13,3,1,105030609
442,100,7410014,2019-03-13,3,2,108030809


In [8]:
def custToProdList(customerId):
    return(mergeOrders.loc[mergeOrders['Customer_ID'] == customerId, 'Product_ID'])

In [9]:
def mostFrequent(customerId):
    return custToProdList(customerId).value_counts().idxmax()

In [10]:
def mostRecent(customerId):
    return custToProdList(customerId).iloc[-1]

In [11]:
def randomPurchase(customerId):
    listRange = len(custToProdList(customerId))
    randNum = random.randrange(0,listRange-1)
    return custToProdList(customerId).iloc[randNum]

In [12]:
def custToProd(customerId,choice):
    if choice == 'frequent':
        return mostFrequent(customerId)
    if choice == 'recent':
        return mostRecent(customerId)
    if choice == 'random':
        return randomPurchase(customerId)

In [13]:
custToProd(9400009,'random')

110060806

In [14]:
custToProd(9400009,'recent')

104030704

In [15]:
custToProd(9400009,'frequent')

110051007

In [16]:
testTable = mergeOrders.drop(['Order_Date','N_Items_x','Order_Item'],axis=1)

In [17]:
allProducts = pd.read_csv('allProducts.csv')

In [18]:
testTable = pd.read_csv('mergedTable.csv')

In [19]:
testTable

,Customer_ID,Product_ID,Frequency
0,310020,103060209,1
1,9400009,201030402,1
2,9400009,103040203,1
3,9400009,101020503,1
4,9400009,101040401,1
...,...,...,...
439,7410014,110010307,1
440,7410014,110060408,1
441,7410014,105030609,1
442,7410014,108030809,1


In [20]:
orderList = []
for i in mergeOrders['Order Number'].unique():
    individualOrder = mergeOrders.loc[mergeOrders['Order Number'] == i, ['Product_ID']]
    orderList.append(individualOrder['Product_ID'].tolist())
orderList

[[103060209],
 [201030402,
  103040203,
  101020503,
  101040401,
  201040303,
  103010301,
  201060202,
  201020401],
 [102020705,
  104010805,
  101030804,
  103020705,
  203050603,
  101040905,
  104040703,
  203050806],
 [204011007, 102050907],
 [102060508, 104030301, 104040201, 302060309, 104040309, 103040201],
 [104060403, 102030305, 103010205, 101040302, 103060404, 203040403],
 [103030701],
 [201040709, 204010508, 103060807, 103020607, 102030609, 102010609, 104040706],
 [102030109, 102061009, 201030110],
 [109060605, 110040806, 110020605, 110050903, 110050806, 104010605, 202020503],
 [203061008, 104041001],
 [102060805, 103050106, 104011008],
 [210010907, 210040907, 104010106],
 [204051007,
  104020807,
  210021005,
  210041007,
  104020805,
  204040908,
  204011007,
  104020907],
 [102020206, 103050206, 103040106],
 [101010708,
  101020706,
  201060705,
  203050606,
  201040708,
  101060608,
  103010508,
  101030708],
 [104030807, 102061008],
 [102060108, 210030206],
 [20203050

In [21]:
result = pd.DataFrame({"Customer_ID":testTable['Customer_ID'],
                      "Product_ID":testTable['Product_ID'],
                      "Frequency":testTable['Frequency']})

result1 = result.pivot_table(index='Customer_ID',columns='Product_ID',values='Frequency').fillna(0)

In [22]:
from surprise import Dataset
from surprise import Reader

In [23]:
result2 = result1.stack()
result2.to_csv('preStackTable.csv',header=True)
result2 = pd.read_csv('preStackTable.csv')
result2 = result2.rename(columns={"0": "Frequency"})

In [24]:
reader = Reader(rating_scale=(0, 1))

In [25]:
data = Dataset.load_from_df(result2[["Customer_ID", "Product_ID", "Frequency"]], reader)

In [26]:
from surprise import KNNWithMeans

# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)

In [27]:
trainingSet = data.build_full_trainset()

algo.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [28]:
prediction = algo.predict(5210019, 103060209)
prediction.est

0.04825035125304308

In [29]:
def custToProdRec(customerID,productID):
    prediction = algo.predict(customerID, productID)
    pred = prediction.est
    return pred

In [30]:
productList = allProducts['Product_ID'].tolist()

In [31]:
def custReco(customerID):
    recos = {}
    for i in productList:
        rating = custToProdRec(customerID,i)
        if rating < 0.99:
            recos[i] = rating
    sortedRecos = {k: v for k, v in sorted(recos.items(), key=lambda item: item[1], reverse=True)}
    return list(sortedRecos)[0:6]

In [32]:
custReco(5210019)

[210051005, 103020805, 106060104, 107060801, 101021007, 106070504]

In [33]:
# FUNCTION TO TAKE IN PRODUCT ID AND OUTPUT RECO PRODUCT ID

productPrePivot = pd.DataFrame({"Product_ID1":allProducts['Product_ID'],
                      "Product_ID2":allProducts['Product_ID'],
                      "Frequency":0})
productPivot = productPrePivot.pivot_table(index='Product_ID1',columns='Product_ID2',values='Frequency').fillna(0)

In [34]:
productTable = productPivot.stack()
productTable.to_csv('productTable.csv',header=True)
productTable = pd.read_csv('productTable.csv')
productTable = productTable.rename(columns={"0": "Frequency"})

In [35]:
productTable

,Product_ID1,Product_ID2,Frequency
0,101010201,101010201,0.0
1,101010201,101010203,0.0
2,101010201,101010204,0.0
3,101010201,101010401,0.0
4,101010201,101010408,0.0
...,...,...,...
337556,309020701,304060809,0.0
337557,309020701,307010406,0.0
337558,309020701,307020803,0.0
337559,309020701,307030706,0.0


In [36]:
productList1 = productTable['Product_ID1'].tolist()
productList2 = productTable['Product_ID2'].tolist()

In [ ]:
# KEEP COMMENTED OUT, LONG FUNCTION TO RUN, FOR LOOPS 337K ROWS

for i in range(0,len(productList1)):
    for n in orderList:
        if productList1[i] in n:
            if productList2[i] in n:
                productTable.loc[(
                    productTable['Product_ID2'] == productList2[i]
                ),'Frequency'] += 1/(len(n))

In [ ]:
productTable

In [ ]:
# NEW ALGO FOR PRODUCT TO PRODUCT

reader2 = Reader(rating_scale=(0, productTable['Frequency'].max()))
data2 = Dataset.load_from_df(productTable[["Product_ID1", "Product_ID2", "Frequency"]], reader2)
algo2 = KNNWithMeans(sim_options=sim_options)

In [ ]:
trainingSet2 = data2.build_full_trainset()

algo2.fit(trainingSet2)

In [ ]:
def prodToProdRec(productID1,productID2):
    prediction = algo2.predict(productID1, productID2)
    pred = prediction.est
    return pred

In [ ]:
def prodReco(productID):
    recos = {}
    for i in productList:
        if productID != i:
            rating = prodToProdRec(productID,i)
            recos[i] = rating
    sortedRecos = {k: v for k, v in sorted(recos.items(), key=lambda item: item[1], reverse=True)}
    return list(sortedRecos)[0:6]

In [ ]:
prodReco(309020701)